In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import LSTM

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [10]:
class FixedModelLSTMProcessor:
    def __init__(self, dates=[]):
        self.dates           = dates
        self.forecasts       = []         # list[pd.DataFrame]
        self.eval_pairs      = []         # list[(forecast_df, truth_df)]
        self.testset         = None
        self.nf              = None
        self.model           = None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df      = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df      = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df          = pd.DataFrame()
        self.unique_ids      = []

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        input_len  = df_train.groupby("unique_id").size().min()
        max_inp    = max(8, input_len - h - 1)
        self.testset = df_test

        def cfg(trial):
            return {
                "input_size"           : trial.suggest_int("input_size", 8, max_inp),
                "encoder_hidden_size"  : trial.suggest_categorical("encoder_hidden_size", [32, 64, 128]),
                "encoder_n_layers"     : trial.suggest_int("encoder_n_layers", 1, 3),
                "context_size"         : trial.suggest_int("context_size", 1, h),
                "decoder_hidden_size"  : trial.suggest_categorical("decoder_hidden_size", [32, 64, 128]),
                "learning_rate"        : trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"           : trial.suggest_categorical("batch_size", [16, 32, 64]),
                "random_seed"          : trial.suggest_int("random_seed", 1, 99999),
                "accelerator"          : "gpu",
                "devices"              : 2,
                "strategy"             : "auto"
            }

        if level:
            model = AutoLSTM(h=h, backend="optuna", loss=MQLoss(level=level), config=cfg)
        else:
            model = AutoLSTM(h=h, backend="optuna", config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        uid_dates = {uid: g["ds"].sort_values().to_numpy()
                     for uid, g in self.testset.groupby("unique_id")}
        T = min(len(d) for d in uid_dates.values())
        s = self.selected_input_size

        rows = []
        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win = dates[t-s:t]; ref = dates[t-1]
                df_u = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in)
                    refs.append((uid, ref))
            if not batch:
                continue

            fcst_batch = self.nf.predict(df=pd.concat(batch)).set_index("ds")

            for uid, ref in refs:
                f  = fcst_batch[fcst_batch["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                rows.append(merged)

        return pd.concat(rows, ignore_index=True)

    def create_display_df(self, merged_df):
        # merged_df = output of sliding_batch_by_index
        records = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="build display"):
            uid = g["unique_id"].iloc[0]
            ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if col.startswith("AutoLSTM"):
                    if "median" in col:
                        q = 0.5
                    elif "-lo-" in col:
                        lvl = int(col.split("-")[-1]); q = (1 - lvl/100)/2
                    elif "-hi-" in col:
                        lvl = int(col.split("-")[-1]); q = 1 - (1 - lvl/100)/2
                    else:
                        continue
                    tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                               col:"Prediction",
                                                               "y":"GT"})
                    tmp["Unique_id"] = uid
                    tmp["Reference Date"] = ref
                    tmp["Quantile"] = q
                    records.append(tmp)
        self.display_df = (pd.concat(records)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        recs = []
        df = self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]; preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            wis = abs(preds[0.5] - gt)  # AE-only shortcut (matches yours)
            recs.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,"WIS":wis})
        self.wis_df = pd.DataFrame(recs)


In [11]:
test_ids = [f"Y_{i}" for i in test_indices]

In [12]:
processor = FixedModelLSTMProcessor()

In [13]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 15:33:44,902] A new study created in memory with name: no-name-a2c28e0e-a695-4455-83f9-5d4f22e24e8f
[rank: 0] Seed set to 13056
/home/jc2wv/.local/lib/python3.11/site-packages/neuralforecast/models/lstm.py:161: UserWarning:

context_size is deprecated and will be removed in future versions.

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type          | Params | Mode 
--------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [14]:
result=[]

In [15]:
processor.selected_input_size

18

In [16]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/39 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   3%|▎         | 1/39 [00:35<22:18, 35.24s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   5%|▌         | 2/39 [01:09<21:15, 34.48s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  95%|█████████▍| 37/39 [21:20<01:09, 34.70s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  97%|█████████▋| 38/39 [21:55<00:34, 34.61s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 39/39 [22:29<00:00, 34.61s/it]


In [17]:
all_results.to_csv('LSTM_concatenated.csv')

In [18]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoLSTM-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 336960/336960 [03:06<00:00, 1811.08it/s]


In [20]:
pointwise_df.to_csv('LSTM_ALL_STATS.csv')

In [15]:
wis_dfs = [wis_table.iloc[i::4].reset_index(drop=True) for i in range(4)]

In [19]:
np.mean(wis_dfs[3]['WIS'].values)

73.43468595144249

In [20]:
wis_table.to_csv('WIS_LSTM.csv')